# Review

## MRO

In [2]:
class A():
    def show(self):
        print('a.show')

class B(A):
    pass

class C(A):
    def show(self):
        print('c.show')

class D(B, C):
    pass

print(D.__mro__)
D().show()

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
c.show


## Descriptor

In [5]:
class StaticMethod(object):
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f

class Foo():
    @StaticMethod
    def bar():
        return 'This is StaticMethod Foo.bar'
    
    # bar = StaticMethod(bar)

print(Foo.bar())

This is StaticMethod Foo.bar


In [7]:
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        def newfunc(*args):
            return self.f(klass, *args)
        return newfunc
    
class Foo():
    @ClassMethod
    def bar(cls, x):
        return 'This is ClassMethod Foo.bar'
    
    # bar = ClassMethod(bar)

print(Foo.bar(3))
print(Foo().bar(3))

This is ClassMethod Foo.bar
This is ClassMethod Foo.bar


In [9]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel)
    
class Foo():
    def __init__(self):
        self._bar = 'baz'
        
    @Property
    def bar(self):
        return self._bar
    
    # bar = Property(bar)
    
    @bar.setter
    def bar(self, new_bar):
        self._bar = new_bar
        
    # bar = bar.setter(bar)
    
foo = Foo()
print(foo.bar)

foo.bar = 'qux'
print(foo.bar)

baz
qux


## Metaclass

In [10]:
class Foo(object):
    bar = 'baz'

print(Foo.bar)

baz


is equivalent to

In [11]:
# type(name, bases, attrs)
Foo = type('Foo', (object,), { 'bar': 'baz' })
print(Foo.bar)

baz


In [38]:
# Emulate enum with metaclass

class MyEnumMetaclass(type):
    def __new__(metacls, cls, bases, classdict):
        my_enum_cls = super().__new__(metacls, cls, bases, classdict)
        enum_members = {}
        for k, v in classdict.items():
            if k.startswith('_') and k.endswith('_'):
                 pass
            else:
                setattr(my_enum_cls, k.upper(), k.upper())
        return my_enum_cls

class MyEnum(metaclass=MyEnumMetaclass):
    a = ()
    b = ()

print(MyEnum.A)
print(MyEnum.B)

A
B


# Class Instantiation

In [1]:
class Foo(object):
    def __init__(self, x, y=0):
        self.x = x
        self.y = y

In [4]:
# foo = Foo(1, y=2)
foo = Foo.__call__(1, y=2)
print(foo)
print(foo.x)
print(foo.y)

1
2


In [28]:
# simulating type's __call__()

class FooMetaclass(type):    
    def __call__(self, *args, **kwargs):
        obj = self.__new__(self, *args, **kwargs)
        if obj is not None:
            obj.__init__(*args, **kwargs)
        return obj

class Foo(metaclass=FooMetaclass):
    def __init__(self, x, y=0):
        self.x = x
        self.y = y

foo = Foo(1, y=2)

print(foo)
print(foo.x)
print(foo.y)

1
2


1. `Foo(*args, **kwargs)` is equivalent to `Foo.__call__(*args, **kwargs)`
2. Since `Foo` is an instance of `type`, `Foo.__call__(*args, **kwargs)` calls `type.__call__(Foo, *args, **kwargs)`
3. `type.__call__(Foo, *args, **kwargs)` calls `type.__new__(Foo, *args, **kwargs)` which returns `obj`
4. `obj` is then initialized by calling `obj.__init__(*args, **kwargs)`
5. `obj` is returned

# Attribute Access

In [7]:
class Foo():
    def __init__(self):
        self.bar = 'baz'
        
foo = Foo()

# foo.bar
print(foo.__getattribute__('bar'))

baz


In [17]:
# simulating __getattribute__()

class Foo():
    def __init__(self):
        self.bar = 'baz'
        
    def __getattr__(self, item):
        return 'qux'
        
    def __my_getattribute__(self, item):
        if item in self.__class__.__dict__:
            v = self.__class__.__dict__[item]
        elif item in self.__dict__:
            v = self.__dict__[item]
        else:
            v = self.__getattr__(item)
        if hasattr(v, '__get__'):
            v = v.__get__(self, type(self))
        return v

foo = Foo()

# foo.bar
print(foo.__my_getattribute__('bar'))

baz


In [19]:
class Foo:
    class_attr = 'class_attr'
    
    def __init__(self):
        self.dict_attr = 'dict_attr'
        
    @property
    def property_attr(self):
        return 'property_attr'
    
    def __getattr__(self, item):
        return '__getattr__'
    
    def __my_getattribute__(self, item):
        if item in self.__class__.__dict__:
            v = self.__class__.__dict__[item]
        elif item in self.__dict__:
            v = self.__dict__[item]
        else:
            v = self.__getattr__(item)
        if hasattr(v, '__get__'):
            v = v.__get__(self, type(self))
        return v
    
foo = Foo()

print(foo.class_attr)
print(foo.dict_attr)
print(foo.property_attr)
print(foo.dynamic_attr)

print()

print(foo.__my_getattribute__('class_attr'))
print(foo.__my_getattribute__('dict_attr'))
print(foo.__my_getattribute__('property_attr'))
print(foo.__my_getattribute__('dynamic_attr'))

class_attr
dict_attr
property_attr
__getattr__

class_attr
dict_attr
property_attr
__getattr__


# Custom ORM

## The goal

In [43]:
class Base():
    def __init__(self, *args, **kwargs):
        pass

class Column():
    def __init__(self, dtype):
        pass

class Relationship():
    def __init__(self, foreign_model_class):
        pass

Make this work:

In [45]:
class Email(Base):
    address = Column(str)
    
class User(Base):
    name = Column(str)
    emails = Relationship('Email')
    
user = User(name='CP')
user.emails = [
    Email(address='cp1@od.hk'),
    Email(address='cp2@od.hk')
]

assert user.name == 'CP'
assert user.emails == [
    { 'address': 'cp1@od.hk' },
    { 'address': 'cp2@od.hk' }
]

## Step 1. Implement column descriptor

In [61]:
class Column():
    def __init__(self, dtype):
        self.dtype = dtype
        self.data = None
    
    def __get__(self, obj, type):
        return self.dtype(self.data)
        
    def __set__(self, obj, value):
        self.data = value

class User():
    name = Column(str)

user1 = User()
user1.name = 'CP1'
print(user1.name)

user2 = User()
user2.name = 'CP2'
print(user2.name)

CP1
CP2


but...

In [62]:
print(user1.name)

CP2


## Step 2. Implement base class

In [96]:
class Column():
    def __init__(self, dtype):
        self.dtype = dtype

class ColumnDescriptor():
    def __init__(self, dtype):
        self.dtype = dtype
        self.data = None
    
    def __get__(self, obj, type):
        return self.dtype(self.data)
        
    def __set__(self, obj, value):
        self.data = value

class Base():
    def __init__(self, **kwargs):
        cls = self.__class__
        
        # create column descriptors from column definitions
        for k, v in cls.__dict__.items():
            if not k.startswith('_') or not k.endswith('_'):
                setattr(self, k, ColumnDescriptor(v.dtype))
        
        # initialize values
        for k, v in kwargs.items():
            setattr(self, k, v)

class User(Base):
    name = Column(str)

user1 = User(name='CP1')
print(user1.name)

user2 = User(name='CP2')
print(user2.name)

user1.name = 'CP1111'
print(user1.name, user2.name)

user2.name = 'CP2222'
print(user1.name, user2.name)

CP1
CP2
CP1111 CP2
CP1111 CP2222


## Step 3. Implement base metaclass

In [136]:
class Column():
    def __init__(self, dtype):
        self.dtype = dtype

class ColumnDescriptor():
    def __init__(self, dtype):
        self.dtype = dtype
        self.data = None
    
    def __get__(self, obj, type):
        return self.dtype(self.data)
        
    def __set__(self, obj, value):
        self.data = value

class Relationship():
    def __init__(self, foreign_model_name):
        self.data = None
    
    def __get__(self, obj, type_):
        return self.data
    
    def __set__(self, obj, col_list):
        self.data = [col.as_dict() for col in col_list]

class BaseMetaclass(type):
    model_registry = {}
    
    def __new__(metacls, cls, bases, classdict):
        if cls == 'Base':
            return super().__new__(metacls, cls, bases, classdict)
        
        klass = super().__new__(metacls, cls, bases, classdict)
        
        metacls.model_registry[cls] = {
            'cls': klass,
            'column_registry': {},
            'relation_registry': {}
        }
        
        for k, v in classdict.items():
            if isinstance(v, Column):
                metacls.model_registry[cls]['column_registry'][k] = v
            elif isinstance(v, Relationship):
                metacls.model_registry[cls]['relation_registry'][k] = v
                
        return klass


class Base(metaclass=BaseMetaclass):    
    def __init__(self, **kwargs):
        cls = self.__class__
        
        # create column descriptors from column definitions
        for k, v in cls.__dict__.items():
            if not k.startswith('_') or not k.endswith('_'):
                if isinstance(v, Column):
                    setattr(self, k, ColumnDescriptor(v.dtype))
        
        # initialize values
        for k, v in kwargs.items():
            setattr(self, k, v)

    def as_dict(self):
        cls = self.__class__
        registry = cls.__class__.model_registry[cls.__name__]
        result = {}
        for col, col_desc in registry['column_registry'].items():
            result[col] = getattr(self, col)
        for rel, rel_desc in registry['relation_registry'].items():
            result[rel] = getattr(self, rel)
        return result

class Email(Base):
    address = Column(str)
            
class User(Base):
    name = Column(str)
    emails = Relationship('Email')

user = User(name='CP')
user.emails = [
    Email(address='cp1@od.hk'),
    Email(address='cp2@od.hk')
]

assert user.name == 'CP'
assert user.emails == [
    { 'address': 'cp1@od.hk' },
    { 'address': 'cp2@od.hk' }
]